# **Computation of PPP Metrics for Ensembles**

**Important metrics:**

- SH sea ice extent/volume
- regional sea ice extent/volume
- regional NPP, PCO2, SSS, SST
- gridpoint-level fields for sea ice concentration, SST, SSS, NPP, PCO2
- sea ice edge position (latitude at which sea ice concentration drops below 15%)

In [1]:
import os
import warnings
warnings.filterwarnings("ignore", message="invalid value encountered in true_divide")
warnings.filterwarnings("ignore", message="Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range")
warnings.filterwarnings("ignore", message="invalid value encountered in reduce")

import xarray as xr
import numpy as np
import math
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2
import hmei.processing as pcs
import hmei.visualization as vis

In [23]:
rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'

reg_masks = xr.open_dataset(write_rootdir+'regional_global_masks.nc')

In [22]:
pcs.dir_inspect(rootdir)

['CN_ENSEMBLE',
 'CTRL',
 'EXT_ENSEMBLE',
 'GRID',
 'HI_ENSEMBLE',
 'MLD_ENSEMBLE',
 'NPP_ENSEMBLE',
 'PCO2SURF_ENSEMBLE',
 'SSS_ENSEMBLE',
 'SST_ENSEMBLE']

In [10]:
xr.open_dataset(rootdir+subdir+'sst_ENS01_neg01_0170_0179.nc')

<xarray.Dataset>
Dimensions:   (time: 120, xt_ocean: 360, yt_ocean: 200)
Coordinates:
  * time      (time) object 0170-01-16 12:00:00 ... 0179-12-16 12:00:00
  * xt_ocean  (xt_ocean) float64 -279.5 -278.5 -277.5 -276.5 ... 77.5 78.5 79.5
  * yt_ocean  (yt_ocean) float64 -81.5 -80.5 -79.5 -78.5 ... 86.5 87.5 88.5 89.5
Data variables:
    sst       (time, yt_ocean, xt_ocean) float32 ...
Attributes:
    filename:                  01700101.ocean_month.nc
    title:                     ESM2M_pi-control_topaz_C3_verona_weddell0170_n...
    grid_type:                 mosaic
    grid_tile:                 1
    history:                   Sun Dec 22 15:08:36 2019: ncks -4 -L 1 -a -O d...
    NCO:                       netCDF Operators version 4.7.5 (Homepage = htt...
    nco_openmp_thread_number:  1

In [38]:
x = xr.open_dataset(rootdir+subdir+'sst_ENS01_neg01_0170_0179.nc')
y = xr.open_dataset(rootdir+subdir+'sst_ENS01_pos01_0170_0179.nc')

ens_range = ['0170_0179', '0022_0031', '0064_0073', '0106_0115', '0232_0241', '0295_0304']

var = 'sst'
rootdir = '/local/projects/so_predict/esm2m_froelicher/'
writedir = '/home/bbuchovecky/storage/so_predict_derived/'
subdir = var.upper()+'_ENSEMBLE/'

for (ens,yrs) in zip(range(1,7), ens_range):
    for sign in ['neg', 'pos']:
        for pert in range(1,21):
            filename = var+'_ENS'+f'{ens:02}'+'_'+sign+f'{pert:02}'+'_'+yrs+'.nc'
            
            run = xr.open_dataset(rootdir+subdir+filename)
            
            ## compute timeseries for each region
            ## using ocean_grid coordinates (area_t) and reg_masks
            
            ## compute anomaly by subtracting climatology
            ## using method with NumPy arrays
            
            ## save these timeseries in a matrix of dimension (nT,nStart,nEns), 
            ## where nT is the number of time samples (nT=12 month/year*10 years=120), 
            ## nStart is the number of start years (6), 
            ## and nEns is the number of ensemble members (40)
            
            ## write this dataset to a NetCDF file
            

In [28]:
for pert in range(1,21):
    print(f'{pert:02}')

01
02
03
04
05
06
07
08
09
10
11
12
13
14
15
16
17
18
19
20


In [21]:
for (ens,yrs) in zip(range(1,7), ens_range):
    print(str(ens)+' '+yrs)

1 0170_0179
2 0022_0031
3 0064_0073
4 0106_0115
5 0232_0241
6 0295_0304
